In [12]:
"""
# Spark CSV → Parquet Processing

## Goal
- GCS에 있는 CSV 데이터를 Spark로 읽는다
- 데이터 구조를 이해한다
- Parquet 포맷으로 변환한다
- 결과를 다시 GCS에 저장한다

## Pipeline
1. Spark Session 초기화
2. GCS 연동 설정
3. GCS CSV 로드
4. 데이터 탐색 (schema / sample)
5. Parquet 변환
6. GCS 업로드
"""

'\n# Spark CSV → Parquet Processing\n\n## Goal\n- GCS에 있는 CSV 데이터를 Spark로 읽는다\n- 데이터 구조를 이해한다\n- Parquet 포맷으로 변환한다\n- 결과를 다시 GCS에 저장한다\n\n## Pipeline\n1. Spark Session 초기화\n2. GCS 연동 설정\n3. GCS CSV 로드\n4. 데이터 탐색 (schema / sample)\n5. Parquet 변환\n6. GCS 업로드\n'

# Spark Session 초기화

In [12]:
import pyspark
pyspark.__version__

'3.5.8'

In [2]:
import os
from pyspark.sql import SparkSession

# 🔹 Hadoop 사용자 설정 (NullPointerException 방지)
os.environ["HADOOP_USER_NAME"] = "spark"

# 기존 SparkSession이 있으면 중지
try:
  spark.stop()
except:
  pass

spark = (
  SparkSession.builder
  .appName("csv_parquet")
  .master("spark://spark-master:7077")

  # 🔹 Ivy 캐시 경로 명시 (절대 경로)
  .config("spark.jars.ivy", "/home/spark/.ivy2")

  # 🔹 GCS FileSystem 연결 (gcs-connector 사용)
  .config(
      "spark.hadoop.fs.gs.impl",
      "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem"
  )
  .config(
      "spark.hadoop.fs.AbstractFileSystem.gs.impl",
      "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS"
  )

  # 🔹 GCS 인증 (서비스 계정)
  .config(
      "spark.hadoop.google.cloud.auth.service.account.enable", "true"
  )
  .config(
      "spark.hadoop.google.cloud.auth.service.account.json.keyfile",
      "/cred/clickstream-sa.json"
  )

  .getOrCreate()
)

# 로그 레벨 설정
spark.sparkContext.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/29 08:24:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1769513763042'),
 ('spark.hadoop.google.cloud.auth.service.account.json.keyfile',
  '/cred/clickstream-sa.json'),
 ('spark.driver.host', 'e1d772bd46f5'),
 ('spark.executor.id', 'driver'),
 ('spark.app.submitTime', '1769513762935'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --

# GCS CSV 로드

In [5]:
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv(
        "gs://clickstream-pipeline-484705-clickstream-data/test/raw/2019-Nov.csv"
    )
)

df.printSchema()
df.show(5)


root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



[Stage 1:>                                                          (0 + 1) / 1]

+--------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:...|      view|   1004775|2053013555631882655|electronics.smart..

In [9]:
df.select("event_time").show(5)

+-------------------+
|         event_time|
+-------------------+
|2019-11-01 00:00:00|
|2019-11-01 00:00:00|
|2019-11-01 00:00:01|
|2019-11-01 00:00:01|
|2019-11-01 00:00:01|
+-------------------+
only showing top 5 rows



# 데이터 탐색 (schema / sample)

In [6]:
df.dtypes

[('event_time', 'string'),
 ('event_type', 'string'),
 ('product_id', 'string'),
 ('category_id', 'string'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'string'),
 ('user_id', 'string'),
 ('user_session', 'string')]

event_time   → timestamp<br>
price        → double <br>
product_id   → bigint  <br> 
category_id  → bigint   <br>
user_id      → bigint<br>

# data Type Casting

In [4]:
from pyspark.sql import types

click_schema = types.StructType([
    types.StructField("event_time", types.TimestampType(), True),
    types.StructField("event_type", types.StringType(), True),
    types.StructField("product_id", types.LongType(), True),
    types.StructField("category_id", types.LongType(), True),
    types.StructField("category_code", types.StringType(), True),
    types.StructField("brand", types.StringType(), True),
    types.StructField("price", types.DoubleType(), True),
    types.StructField("user_id", types.LongType(), True),
    types.StructField("user_session", types.StringType(), True),
])


In [5]:
df = (
    spark.read
    .option("header", "true")
    .schema(click_schema)
    .csv(
        "gs://clickstream-pipeline-484705-clickstream-data/test/raw/2019-Nov.csv"
    )
)

df.printSchema()
df.show(5)


root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_session: string (nullable = true)



+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.smart...|xiaomi

In [13]:
from pyspark.sql import types                                                                                             
from pyspark.sql.functions import col, to_timestamp, regexp_replace, monotonically_increasing_id                          
from pyspark.sql.functions import when, lit, concat_ws, array, to_date, date_format         
                                                                                                                        
# ===========================                                                                                             
# 경로 설정                                                                                                               
# ===========================                                                                                             
# INPUT: GCS                                                                                                              
GCS_BUCKET = "clickstream-pipeline-484705-clickstream-data"                                                               
INPUT_PATH = f"gs://{GCS_BUCKET}/test/raw/2019-Nov.csv"                                                                   
                                                                                                                        
# OUTPUT: 로컬 (컨테이너 → 호스트 매핑됨)                                                                                 
LOCAL_BASE = "/home/spark/work/output"                                                                                    
                                                                                                                        
OUTPUT_PATHS = {                                                                                                          
  "bronze_invalid": f"{LOCAL_BASE}/bronze/invalid/2019-Nov/",                                                           
  "silver": f"{LOCAL_BASE}/silver/clickstream/2019-Nov/",                                                               
}                                                                                                                         
                                                                                                                        
print(f"📂 입력:  {INPUT_PATH}")                                                                                          
print(f"📂 출력 (호스트 경로):")                                                                                          
print(f"   Invalid: ./spark/output/bronze/invalid/2019-Nov/")                                                             
print(f"   Silver:  ./spark/output/silver/clickstream/2019-Nov/")                                                         
print() 

📂 입력:  gs://clickstream-pipeline-484705-clickstream-data/test/raw/2019-Nov.csv
📂 출력 (호스트 경로):
   Invalid: ./spark/output/bronze/invalid/2019-Nov/
   Silver:  ./spark/output/silver/clickstream/2019-Nov/



In [14]:
# ===========================                                                                                             
# 1️⃣ Bronze 스키마 (전부 String)                                                                                          
# ===========================                                                                                             
bronze_schema = types.StructType([                                                                                        
  types.StructField("event_time", types.StringType(), True),                                                            
  types.StructField("event_type", types.StringType(), True),                                                            
  types.StructField("product_id", types.StringType(), True),                                                            
  types.StructField("category_id", types.StringType(), True),                                                           
  types.StructField("category_code", types.StringType(), True),                                                         
  types.StructField("brand", types.StringType(), True),                                                                 
  types.StructField("price", types.StringType(), True),                                                                 
  types.StructField("user_id", types.StringType(), True),                                                               
  types.StructField("user_session", types.StringType(), True),                                                          
]) 

In [15]:
# ===========================                                                                                             
# 2️⃣ GCS에서 Bronze 로드                                                                                                  
# ===========================                                                                                             
print(f"📥 GCS에서 데이터 읽는 중...")                                                                                    
                                                                                                                        
df_bronze = (                                                                                                             
  spark.read                                                                                                            
  .option("header", "true")                                                                                             
  .schema(bronze_schema)                                                                                                
  .csv(INPUT_PATH)                                                                                                      
)                                                                                                                         
                                                                                                                        
total_records = df_bronze.count()                                                                                         
print(f"✅ 읽기 완료: {total_records:,} 레코드\n")     

📥 GCS에서 데이터 읽는 중...


[Stage 10:=======================================================>(67 + 1) / 68]

✅ 읽기 완료: 67,501,979 레코드



In [18]:
# ===========================                                                                                             
# 3️⃣ 타입 캐스팅 (5개 컬럼 전부)                                                                                          
# ===========================     
print("🔄 타입 캐스팅 중...")      
                                                                                                                        
df_bronze_typed = (
    df_bronze
    .withColumn("_row_id", monotonically_increasing_id())

    # 1. event_time
    .withColumn("event_time_raw", col("event_time"))
    .withColumn(
        "event_time_typed",
        to_timestamp(
            regexp_replace(col("event_time"), " UTC$", ""),
            "yyyy-MM-dd HH:mm:ss"
        )
    )

    # 2. product_id
    .withColumn("product_id_raw", col("product_id"))
    .withColumn("product_id_typed", col("product_id").cast("long"))

    # 3. category_id
    .withColumn("category_id_raw", col("category_id"))
    .withColumn("category_id_typed", col("category_id").cast("long"))

    # 4. price
    .withColumn("price_raw", col("price"))
    .withColumn("price_typed", col("price").cast("double"))

    # 5. user_id
    .withColumn("user_id_raw", col("user_id"))
    .withColumn("user_id_typed", col("user_id").cast("long"))
) 


🔄 타입 캐스팅 중...


In [19]:
# ===========================                                                                                             
# 4️⃣ 캐스팅 실패 검증                                                                                                     
# ===========================     
import time
print("🔍 캐스팅 실패 검증 중...")     
                                                                     
start_time = time.time() 

df_cast_fail = df_bronze_typed.filter(                                                                                    
  (col("event_time_raw").isNotNull() & col("event_time_typed").isNull()) |                                                    
  (col("product_id_raw").isNotNull() & col("product_id_typed").isNull()) |                                                    
  (col("category_id_raw").isNotNull() & col("category_id_typed").isNull()) |                                                  
  (col("price_raw").isNotNull() & col("price_typed").isNull()) |                                                              
  (col("user_id_raw").isNotNull() & col("user_id_typed").isNull())                                                            
)                                                                                                                                                                                              
                                                                                                     
cast_fail_count = df_cast_fail.count()           

end_time = time.time()  # ⏱️ 종료                                                                  
elapsed = end_time - start_time 

cast_fail_rate = cast_fail_count / total_records if total_records > 0 else 0                                              
                                                                                                                        
print(f"\n{'='*50}")                                                                                                      
print(f"📊 타입 캐스팅 검증 결과")                                                                                        
print(f"{'='*50}")                                                                                                        
print(f"✅ 총 레코드:      {total_records:,}")                                                                            
print(f"⚠️  캐스팅 실패:   {cast_fail_count:,} ({cast_fail_rate:.4%})")                                                   
print(f"{'='*50}\n")                                  

print(f"⏱️  소요 시간:     {elapsed:.2f}초")  
                                  

🔍 캐스팅 실패 검증 중...


[Stage 13:=======================================================>(67 + 1) / 68]


📊 타입 캐스팅 검증 결과
✅ 총 레코드:      67,501,979
⚠️  캐스팅 실패:   0 (0.0000%)

⏱️  소요 시간:     766.99초


In [ ]:
# ===========================                                                                                             
# 5️⃣ 실패 데이터 로컬 저장                                                                                                
# ===========================                                                                                             
if cast_fail_count > 0:                                                                                                   
  print(f"💾 Invalid 데이터 로컬 저장 중...")                                                                           
                                                                                                                        
  df_cast_fail_labeled = df_cast_fail.withColumn(                                                                       
      "failure_reason",                                                                                                 
      concat_ws(", ",                                                                                                   
          array(                                                                                                        
              when(col("event_time_raw").isNotNull() & col("event_time_typed").isNull(),                                      
                   lit("event_time")).otherwise(lit("")),                                                               
              when(col("product_id_raw").isNotNull() & col("product_id_typed").isNull(),                                      
                   lit("product_id")).otherwise(lit("")),                                                               
              when(col("category_id_raw").isNotNull() & col("category_id_typed").isNull(),                                    
                   lit("category_id")).otherwise(lit("")),                                                              
              when(col("price_raw").isNotNull() & col("price_typed").isNull(),                                                
                   lit("price")).otherwise(lit("")),                                                                    
              when(col("user_id_raw").isNotNull() & col("user_id_typed").isNull(),                                            
                   lit("user_id")).otherwise(lit(""))                                                                   
          )                                                                                                             
      )                                                                                                                 
  )                                                                                                                     
                                                                                                                        
  df_cast_fail_labeled.select(                                                                                          
      "event_time_raw", "event_type", "product_id_raw", "category_id_raw",                                              
      "category_code", "brand", "price_raw", "user_id_raw", "user_session",                                             
      "failure_reason"                                                                                                  
  ).write.mode("overwrite").parquet(OUTPUT_PATHS["bronze_invalid"])                                                     
                                                                                                                        
  print(f"✅ Invalid 저장 완료\n")                                                                                      
else:                                                                                                                     
  print(f"✅ 캐스팅 실패 없음\n")       

In [ ]:
# ===========================                                                                                             
# 6️⃣ Silver: 성공 데이터만                                                                                                
# ===========================  
# 비즈니스 로직상 필수 컬럼만 체크 
print(f"🔄 Silver 데이터 생성 중...")                                                                                     
                                                                                                                        
df_silver = (                                                                                                             
  df_bronze_typed                                                                                                       
  .filter(                                                                                                              
      col("event_time_typed").isNotNull() &                                                                                   
      col("user_id_typed").isNotNull() &                                                                                      
      col("product_id_typed").isNotNull()                                                                                     
  )                                                                                                                     
    .select(                                                                                           
      col("event_time_typed").alias("event_time"),                                                   
      col("event_type"),                                                                             
      col("product_id_typed").alias("product_id"),                                                   
      col("category_id_typed").alias("category_id"),                                                 
      col("category_code"),                                                                          
      col("brand"),                                                                                  
      col("price_typed").alias("price"),                                                             
      col("user_id_typed").alias("user_id"),                                                         
      col("user_session")                                                                            
  )   
)                                                                                                                         
                                                                                                                        
silver_count = df_silver.count()                                                                                          
silver_rate = silver_count / total_records if total_records > 0 else 0                                                    
                                                                                                                        
print(f"✅ Silver 레코드: {silver_count:,} ({silver_rate:.2%})")     

In [ ]:
# ===========================                                                                                             
# 7️⃣ Silver 로컬 저장 (파티션)                                                                                            
# ===========================                                                                                             
print(f"💾 Silver 로컬 저장 중...")                                                                                       
                                                                                                                        
df_silver_partitioned = (                                                                                                 
  df_silver                                                                                                             
  .withColumn("event_date", to_date(col("event_time")))                                                                 
  .withColumn("event_month", date_format(col("event_time"), "yyyy-MM"))                                                 
)                                                                                                                         
                                                                                                                        
(
  df_silver_partitioned.write
  .mode("overwrite")
  .partitionBy("event_month", "event_date")
  .option("compression", "snappy")
  .parquet(OUTPUT_PATHS["silver"])
)
                                                                                                                        
print(f"✅ Silver 저장 완료\n")                                                                                           
                                       

In [ ]:
# ===========================                                                                                             
# 8️⃣ 저장 확인                                                                                                            
# ===========================                                                                                             
print("📁 저장된 파일 확인:")                                                                                             
print("\nInvalid 데이터:")                                                                                                
!ls -lh {OUTPUT_PATHS["bronze_invalid"]} 2>/dev/null || echo "  (없음)"                                                   
                                                                                                                        
print("\nSilver 데이터:")                                                                                                 
!ls -lh {OUTPUT_PATHS["silver"]}                                                                                          
                                                                                                                        
print("\nSilver 파티션 구조:")                                                                                            
!find {OUTPUT_PATHS["silver"]} -type d -name "event_date=*" | head -5                                                     
                                                                              

In [12]:
# ===========================                                                                                             
# 9️⃣ 최종 요약                                                                                                            
# ===========================                                                                                             
print(f"\n{'='*50}")                                                                                                      
print(f"📝 최종 요약")                                                                                                    
print(f"{'='*50}")                                                                                                        
print(f"📥 입력 (GCS):  {INPUT_PATH}")                                                                                    
print(f"📦 Bronze:      {total_records:,} 레코드")                                                                        
print(f"⚠️  Invalid:    {cast_fail_count:,} 레코드 ({cast_fail_rate:.4%})")                                               
print(f"✅ Silver:      {silver_count:,} 레코드 ({silver_rate:.2%})")                                                     
print(f"\n📤 출력 (로컬):")                                                                                               
print(f"   Invalid: ./spark/output/bronze/invalid/2019-Nov/")                                                             
print(f"   Silver:  ./spark/output/silver/clickstream/2019-Nov/")                                                         
print(f"{'='*50}\n")

📂 입력:  gs://clickstream-pipeline-484705-clickstream-data/test/raw/2019-Nov.csv
📂 출력 (호스트 경로):
   Invalid: ./spark/output/bronze/invalid/2019-Nov/
   Silver:  ./spark/output/silver/clickstream/2019-Nov/

📥 GCS에서 데이터 읽는 중...


✅ 읽기 완료: 67,501,979 레코드

🔄 타입 캐스팅 중...
🔍 캐스팅 실패 검증 중...


26/01/29 10:22:30 WARN TaskSetManager: Lost task 3.0 in stage 9.0 (TID 144) (172.18.0.5 executor 0): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '2019-11-03 01:28:51 UTC' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:142)
	at org.apache.spark.sql.c

Py4JJavaError: An error occurred while calling o143.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 4 times, most recent failure: Lost task 0.3 in stage 9.0 (TID 162) (172.18.0.5 executor 0): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '2019-11-01 00:00:00 UTC' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.sp

# 타입 캐스팅 검증 코드 

# Parquet 변환

In [7]:
from pyspark.sql.functions import col, to_timestamp, to_date, date_format

df_parquet = (
  df
  .withColumn("event_time", to_timestamp(col("event_time"), "yyyy-MM-dd HH:mm:ss z"))
  .withColumn("event_date", to_date(col("event_time")))
  .withColumn("event_month", date_format(col("event_time"), "yyyy-MM"))
  .withColumn("product_id", col("product_id").cast("long"))
  .withColumn("category_id", col("category_id").cast("long"))
  .withColumn("price", col("price").cast("double"))
  .withColumn("user_id", col("user_id").cast("long"))
)

(
  df_parquet
  .write
  .mode("overwrite")
  .partitionBy("event_month", "event_date")
  .option("compression", "snappy")
  .parquet("/home/spark/work/output/parquet")
)

print(f"✅ Saved to: /home/spark/work/output/parquet")                                                                                                                        
print(f"   Host path: ./spark/output/parquet")  

[Stage 3:========================================================>(67 + 1) / 68]

✅ Saved to: /home/spark/work/output/parquet
   Host path: ./spark/output/parquet


In [ ]:
#셀확인
!ls -lh /home/spark/work/output/parquet 

In [ ]:
!du -sh /home/spark/work/output/parquet

# GCS 업로드

In [3]:
spark.sparkContext.uiWebUrl

'http://f40ced364137:4040'